## Para aquellos negocios que tengan más de 5 sucursales y su distribución probabilística de sucursales por ciudad. Calcule la divergencia de Kullback Leibler entre todos los negocios:
* ¿Por qué dan casi todas infinito? Para corregirlo recalcule todo sumando 1*10^-10 en el denominador del logaritmo.
* ¿A qué negocios y en qué sentido corresponde la menor divergencia de Kullback Leibler? (recordar que la divergencia es dirigida)
* ¿Cuál es el negocio con mejor divergencia de Kullback-Leibler respecto de Burger King? ¿Y de McDonalds?

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math

dataset_path = './dataset'
b_df = pd.read_csv(dataset_path + '/business.csv')
b_df =b_df.sort_values(by='business_id')
b_df.name.value_counts().sort_values(ascending=False) # cant sucursales
b_df['city'] = b_df['city'].apply(lambda ci: ci.strip().lower())
b_df['name'] = b_df['name'].apply(lambda na: na.strip().lower())
b_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
29889,---kPU91CF4Lq2-WlRu9Lw,frankie's raw bar,4903 State Rd 54,new port richey,FL,34652,28.217288,-82.733344,4.5,24.0,1,"{'Alcohol': ""u'none'"", 'OutdoorSeating': 'True...","Seafood, Restaurants, Latin American, Food, Fo...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
118535,--0iUa4sNDFiZFrAdIWhZQ,pupuseria y restaurant melba,6 S White Horse Pike,clementon,NJ,08021,39.817850,-74.993364,3.0,2.0,1,"{'Alcohol': ""u'none'"", 'RestaurantsDelivery': ...","Food, Restaurants, Specialty Food, Mexican, Et...",NaN
46313,--30_8IhuyMHbSOcNWd6DQ,action karate,2235 York Rd,jamison,PA,18929,40.255362,-75.088399,3.5,4.0,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Trainers, Active Life, Fitness & Instruction, ...",NaN
45464,--7PUidqRWpRSpXebiyxTg,humpty's family restaurant,9910 108A Avenue,edmonton,AB,T5H,53.554659,-113.493040,2.0,4.0,0,"{'GoodForKids': 'True', 'RestaurantsAttire': ""...","Breakfast & Brunch, Restaurants",NaN
132722,--7jw19RH9JKXgFohspgQw,mcilwain family dentistry & ahrens orthodontics,"26908 Foggy Creek Rd, Ste 101",wesley chapel,FL,33544,28.189156,-82.371421,4.0,5.0,1,"{'ByAppointmentOnly': 'True', 'BusinessAccepts...","Orthodontists, Health & Medical, General Denti...","{'Monday': '7:0-17:30', 'Tuesday': '7:0-17:30'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106478,zznZqH9CiAznbkV6fXyHWA,que pasta nola,500 S Telemachus St,new orleans,LA,70119,29.968303,-90.099917,5.0,12.0,1,"{'WheelchairAccessible': 'True', 'Caters': 'Tr...","Pop-Up Restaurants, Cuban, Restaurants, Honduran","{'Monday': '16:0-21:0', 'Wednesday': '16:0-22:..."
3323,zztOG2cKm87I6Iw_tleZsQ,project management academy,240 Mall Blvd,king of prussia,PA,19406,40.092606,-75.393004,5.0,5.0,1,NaN,"Test Preparation, Specialty Schools, Adult Edu...","{'Monday': '8:30-20:30', 'Tuesday': '8:30-20:3..."
93163,zzu6_r3DxBJuXcjnOYVdTw,cafe diblasi,"1801 Stumpf Blvd, Ste 4",gretna,LA,70056,29.911358,-90.035558,3.5,0.0,0,"{'OutdoorSeating': 'False', 'Alcohol': ""u'full...","Restaurants, Cafes, Italian",NaN
75076,zzw66H6hVjXQEt0Js3Mo4A,sullivan farms christmas trees,14614 Manchester Rd,ballwin,MO,63011,38.592897,-90.527920,3.5,4.0,1,"{'BusinessParking': ""{'garage': False, 'street...","Nurseries & Gardening, Home Services, Shopping...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."


In [2]:
b_df.city

29889     new port richey
118535          clementon
46313             jamison
45464            edmonton
132722      wesley chapel
               ...       
106478        new orleans
3323      king of prussia
93163              gretna
75076             ballwin
6729         philadelphia
Name: city, Length: 150346, dtype: object

In [4]:
cities = b_df.city
cities = cities.value_counts().index.tolist()

In [5]:
names = b_df.name
suc = names.value_counts()#.sort_values(ascending=False)
suc = suc[suc > 5]
suc = suc.index.tolist()
b_df2 = b_df
b_df2 = b_df2[b_df2['name'].isin(suc)] #dataframe de los negocios que tienen mas de 5 sucursales
b_df2

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
82286,--N9yp3ZWqQIm7DqKRvorg,firestone complete auto care,6291 9th St N,st petersburg,FL,33702,27.829290,-82.646870,2.5,14.0,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Automotive, Auto Repair, Auto Parts & Supplies...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-19:0', '..."
38991,--SJXpAa0E-GCp2smaHf0A,winn dixie,10667 Big Bend Rd,riverview,FL,33579,27.791333,-82.332240,2.5,10.0,1,"{'BikeParking': 'False', 'RestaurantsPriceRang...","Grocery, Beer, Wine & Spirits, Food","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."
2188,--_9CAxgfXZmoFdNIRrhHA,usps,3135 1st Ave N,st. petersburg,FL,33730,27.772845,-82.676156,3.5,4.0,1,NaN,"Post Offices, Public Services & Government",NaN
27825,--_lZuj_WCGnDG6n0emlRg,ross dress for less,700 Haddonfield Berlin Rd,voorhees,NJ,08043,39.850059,-74.979995,2.0,9.0,1,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","Shopping, Women's Clothing, Discount Store, Ho...","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
141306,--epgcb7xHGuJ-4PUeSLAw,manhattan bagel,200 York Road,willow grove,PA,19090,40.145054,-75.116293,3.0,19.0,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Restaurants, Food, Bagels, Sandwiches, Breakfa...","{'Monday': '7:0-14:0', 'Tuesday': '7:0-14:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13162,zzIF9qp2UoHN48EeZH_IDg,domino's pizza,40188 US Hwy 19 N,tarpon springs,FL,34689,28.137368,-82.740846,3.0,11.0,1,"{'RestaurantsReservations': 'False', 'Ambience...","Italian, Restaurants, Pizza, Chicken Wings, Sa...","{'Monday': '10:30-1:0', 'Tuesday': '10:30-1:0'..."
75539,zzXRdzrVhfNWPHD2MeyWeA,royal farms,1780 Sumneytown Pike,lansdale,PA,19446,40.245754,-75.342402,2.0,15.0,1,"{'BusinessParking': ""{'garage': False, 'street...","Restaurants, Coffee & Tea, Food, Chicken Wings...",NaN
18333,zziDpuuJw-Km1J4BaGpBKA,honey baked ham company,6936 S Lindbergh Blvd,st. louis,MO,63125,38.509917,-90.333968,3.5,5.0,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsRes...","Sandwiches, Event Planning & Services, Food, D...","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
26920,zznJox6-nmXlGYNWgTDwQQ,dunkin',23924 US Highway 19 N,clearwater,FL,33765,27.990058,-82.730226,1.5,21.0,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Food, Restaurants, Coffee & Tea, Donuts","{'Monday': '5:0-21:0', 'Tuesday': '5:0-21:0', ..."


In [6]:
df3 = pd.DataFrame()
df3['name'] = b_df2['name']
df3['city'] = b_df2['city']

def foo(name, data):
    di = {}
    df = pd.DataFrame()
    df['city'] = data['city']
    df = data[data['name'].isin([name])]
    df= df.drop('name', axis=1)
    pr = df.city.value_counts().map(lambda x: x/df.city.size)
    dat = []
    for r in df.values:
        aux = r
        aux = np.insert(aux,1,pr[r[0]])
        dat.append(aux)
    res = pd.DataFrame(dat, columns=['city', 'prob'])
    res =res.drop_duplicates(subset='city')
    di[name] = res
    return di

foo("starbucks", df3)

{'starbucks':              city      prob
 0       st. louis  0.035912
 1        franklin  0.008287
 2    philadelphia  0.078729
 3        deptford  0.002762
 4      jenkintown  0.001381
 ..            ...       ...
 699  willow grove  0.001381
 703       evesham  0.001381
 704   penns grove  0.001381
 716         eagle  0.001381
 717   maple shade  0.001381
 
 [197 rows x 2 columns]}

In [8]:
df = pd.DataFrame()
df['name'] = b_df2['name']
df['city'] = b_df2['city']

di = {}
for s in suc:
    di.update(foo(s, df))
di


{'starbucks':              city      prob
 0       st. louis  0.035912
 1        franklin  0.008287
 2    philadelphia  0.078729
 3        deptford  0.002762
 4      jenkintown  0.001381
 ..            ...       ...
 699  willow grove  0.001381
 703       evesham  0.001381
 704   penns grove  0.001381
 716         eagle  0.001381
 717   maple shade  0.001381
 
 [197 rows x 2 columns],
 "mcdonald's":                city      prob
 0       cherry hill  0.004267
 1       new orleans  0.019915
 2         nashville  0.031294
 3      indianapolis  0.068279
 6            camden  0.002845
 ..              ...       ...
 687     coatesville  0.001422
 691  fairless hills  0.001422
 693         trevose  0.001422
 697        deptford  0.001422
 701     old hickory  0.001422
 
 [263 rows x 2 columns],
 "dunkin'":                   city      prob
 0       temple terrace  0.003922
 1         apollo beach  0.001961
 2              marlton  0.005882
 3          maple shade  0.001961
 4          mount 

In [18]:
di.keys().__len__()

1137